# Topic modeling

Zadanie proszę wykonać w parach, ale... na dwóch komputerach. Na obu komputerach proszę najpierw wykonać instrukcje z sekcji **Przygotowanie**. Następnie na jednym komputerze proszę od razu odkomentować i uruchomić kod w sekcji **Wikipedia** i **Wizualizacja**. Na drugim komputerze proszę rozwiązać zadania z sekcji **Tweety** i **Wizualizacja**.

Po wykonaniu tego zadania powinieneś:
+ potrafić wykonać podstawowy topic modeling,
+ umieć stworzyć słownik mapujący identyfikatory na słowa,
+ potrafić stworzyć macierz wektorów TF-IDF,
+ wiedzieć jak wykorzystać LDA do określenia proporcji tematów w nowym dokumencie tekstowym,
+ potrafić zwizualizować wyniki algorytmu LDA.

Do wykonania zadania wykorzystamy bibliotekę [gensim](https://radimrehurek.com/gensim/), która oferuje szereg metod do analizy tekstu. Warto w wolnej chwili zobaczyć co oprócz algorytmu LDA zostało zaimplementowane w ramach tego modułu!

## Przygotowanie

Aby zadziałała wizualizacja, musimy najpierw zauktualizować bibliotekę `scipy` i doinstalować bibliotekę `pyldavis`. Będzie to okazja, żeby zobaczyć jak zarządza się bibliotekami w anacondzie. Jeśli kogoś interesuje co oznaczają kolejne komendy, proszę zajrzeć do [dokumentacji Anacondy](http://conda.pydata.org/docs/using/pkgs.html).

1. Zatrzymaj kernel (serwer jupyter notebooka)
2. Otwórz terminal
3. Wpisz `conda update scipy` (i Enter gdy spytają `Proceed`)
4. Popatrz na paski postępu
5. Wpisz `activate root`
6. Wpisz `pip install pyldavis`
7. Wpisz `deactivate`

Koniec. Możesz ponownie odpalić notatnik i przejść do kolejnych kroków.

## Wikipedia

Ten fragemnt kodu stanowi przykład uruchomienia topic modelingu na większym zbiorze danych. Ponieważ wyliczenie modelu będzie trwać od kilku do kilkunastu minut, niech każda para uruchomi ten przykład tylko na jednym komputerze.

Aby uruchomić przykład, w folderze z notatnikiem muszą znajdować się pliki `wiki_wordids.txt.bz2` i `wiki_tfidf.mm` ściągnięte wraz z notatnikiem. Przykład zbudowany jest w oparciu o podzbiór stron wikipedii dostępny pod adresem: https://dumps.wikimedia.org/enwiki/latest/. Strony zostały przekonwertowane na reprezentację wektorową za pomocą skryptu:
`python -m gensim.scripts.make_wiki`.

**Przeczytaj komentarze zanim uruchomisz kod.**

In [1]:
import logging
import gensim

# Wyłączyłem logowanie, bo wyjście Notebooka po wyrenderowaniu do HTMLa wygląda strasznie i jest nieczytlene
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)

# Odczytujemy z pliku mapowanie/słownik id->słowo
id2word = gensim.corpora.Dictionary.load_from_text('wiki_wordids.txt.bz2')

# Odczytujemy z pliku reprezentację wektorową korpusu (macierz wetkorów TF-IDF)
mm = gensim.corpora.MmCorpus('wiki_tfidf.mm')
print(mm)

# Tworzymy model LDA z 20 grupami wykonując 20 iteracji na całym zbiorze
lda = gensim.models.LdaMulticore(corpus=mm, id2word=id2word, num_topics=20, passes=20, workers=4)
# Alternatywnie w razie problemów z wielowątkowością:
# lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=20, update_every=0, passes=20)
lda.print_topics(20)

2021-01-13 21:07:03,041 : WARNING : Text does not contain num_docs on the first line.


MmCorpus(4672 documents, 19960 features, 1762996 non-zero entries)


[(0,
  '0.002*"parliament" + 0.002*"election" + 0.002*"treaty" + 0.002*"egypt" + 0.002*"democratic" + 0.002*"elections" + 0.002*"el" + 0.002*"jpg" + 0.002*"census" + 0.002*"denmark"'),
 (1,
  '0.004*"beer" + 0.003*"antarctic" + 0.003*"batter" + 0.002*"aa" + 0.002*"barnard" + 0.002*"tropical" + 0.002*"cola" + 0.002*"batting" + 0.002*"rum" + 0.002*"abba"'),
 (2,
  '0.001*"banach" + 0.001*"albanian" + 0.001*"albania" + 0.001*"disco" + 0.001*"cantonese" + 0.001*"ethylene" + 0.001*"aeneas" + 0.001*"asimov" + 0.001*"lara" + 0.001*"borneo"'),
 (3,
  '0.003*"poems" + 0.003*"constantine" + 0.003*"pope" + 0.002*"poetry" + 0.002*"poem" + 0.002*"saxon" + 0.002*"monastery" + 0.002*"di" + 0.002*"caesar" + 0.002*"bavaria"'),
 (4,
  '0.004*"cell" + 0.003*"acid" + 0.003*"cells" + 0.003*"carbon" + 0.002*"dna" + 0.002*"plants" + 0.002*"magnitude" + 0.002*"genus" + 0.002*"electron" + 0.002*"stars"'),
 (5,
  '0.006*"christ" + 0.005*"jesus" + 0.004*"theorem" + 0.004*"bible" + 0.004*"algebra" + 0.003*"church

## Tweety

W parach stwórzcie model LDA w oparciu o załączone Tweety. W tym celu należy przekonwertować pliki tekstowe na reprezentację wektorową zapisując wcześniej mapowanie id->słowo w postaci słownika. Opis jak stworzyć wymienione struktury danych można znaleźć na stronie: https://radimrehurek.com/gensim/tut1.html.

**Zad. 1: Wczytaj tweety z pliku tweets.tsv do zmiennej `tweets`.**

In [2]:
import gensim
import logging
import nltk
import re
import pandas as pd
from collections import Counter

stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
            "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
            "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
            "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've",
            "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "of",
            "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "shan't", "she", "she'd",
            "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs",
            "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're",
            "they've", "this", "those", "through", "to", "until", "up", "very", "was", "wasn't", "we", "we'd",
            "we'll", "we're", "we've", "were", "weren't", "what", "what's", "when", "when's", "where", "where's",
            "which", "while", "who", "who's", "whom", "with", "would", "you", "you'd", "you'll", "you're", "you've",
            "your", "yours", "yourself", "yourselves", "above", "again", "against", "aren't", "below", "but", "can't",
            "cannot", "couldn't", "didn't", "doesn't", "don't", "down", "few", "hadn't", "hasn't", "haven't", "if",
            "isn't", "mustn't", "no", "nor", "not", "off", "out", "over", "shouldn't", "same", "too", "under", "why",
            "why's", "won't", "wouldn't"]

tweets = pd.read_csv('tweets.tsv', sep='\t', header=None)

**Zad. 2: Dokonaj tokenizacji słów, usuń te z stoplisty (`stopwords`) oraz występujące tylko raz. Wynik przypisz do zmiennej `texts`.**

In [3]:
texts = [[word for word in document.lower().split() if word not in stopwords]
         for document in tweets[2]]

print("Before removing one-time words")
for text in texts[:3]:
    print(text)

counter = Counter()
for document in texts:
    counter.update(document)

texts = [[word for word in document if counter[word] > 1]
         for document in texts]

print("\nAfter removing one-time words")
for text in texts[:3]:
    print(text)

Before removing one-time words
['dear', '@microsoft', 'newooffice', 'mac', 'great', 'all,', 'lync', 'update?', "c'mon."]
['@microsoft', 'make', 'system', 'eat', 'friggin', 'discs.', '2nd', 'time', 'happened', 'sick', 'it!']
['available']

After removing one-time words
['dear', '@microsoft', 'mac', 'great', 'all,', "c'mon."]
['@microsoft', 'make', 'system', 'eat', '2nd', 'time', 'happened', 'sick', 'it!']
['available']


**Zad. 3: Stwórz słownik id->słowo i przypisz do zmiennej `id2word`.**

In [4]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

id2word = Dictionary(texts)
print(id2word)

Dictionary(5768 unique tokens: ['@microsoft', 'all,', "c'mon.", 'dear', 'great']...)


**Zad. 4: Stwórz reprezentację wektorową korpusu (macierz wektorów TF-IDF), wynik przypisz do zmiennej `mm`.**

In [5]:
mm = [id2word.doc2bow(text) for text in texts]

**Zad. 5: Odpal poniższy kod i odkryj 10 tematów za pomocą algorytmu LDA. Jeśli masz czas, zwiększ wartości parametrów num_topics i passes przy tworzeniu modelu LDA, i sprawdź jak to wpłynie na rezultat.**

In [6]:
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=10, update_every=0, passes=20)
# alternatywnie lda = gensim.models.LdaMulticore(corpus=mm, id2word=id2word, num_topics=10, passes=20)
lda.print_topics(10)

[(0,
  '0.018*"-" + 0.013*"conor" + 0.012*"mcgregor" + 0.012*"watch" + 0.012*"will" + 0.009*"&amp;" + 0.008*"new" + 0.007*"harry" + 0.007*"may" + 0.007*"potter"'),
 (1,
  '0.032*"may" + 0.018*"will" + 0.016*"trump" + 0.015*"donald" + 0.014*"jeb" + 0.013*"-" + 0.012*"bush" + 0.012*"joe" + 0.012*"biden" + 0.011*"1st"'),
 (2,
  '0.016*"see" + 0.016*"mike" + 0.016*"magic" + 0.013*"hannibal" + 0.012*"chris" + 0.011*"xxl" + 0.011*"brown" + 0.011*"tomorrow" + 0.011*"going" + 0.007*"5th"'),
 (3,
  '0.023*"going" + 0.021*"tomorrow" + 0.020*"may" + 0.018*"galaxy" + 0.016*"note" + 0.015*"5" + 0.013*"day" + 0.010*"disneyland" + 0.008*"will" + 0.008*"grateful"'),
 (4,
  '0.034*"amazon" + 0.027*"prime" + 0.020*"day" + 0.019*"may" + 0.013*"1st" + 0.013*"black" + 0.008*"&amp;" + 0.007*"get" + 0.007*"friday" + 0.006*"ios"'),
 (5,
  '0.025*"just" + 0.014*"bob" + 0.013*"may" + 0.012*"marley" + 0.011*"bobby" + 0.010*"jindal" + 0.010*"-" + 0.009*"day" + 0.009*"1st" + 0.008*"game"'),
 (6,
  '0.030*"apple" +

#### Zwiększenie `passes` do 50
Zwiększenie passess do 50 spowodowało że algorytm działa dłużej i zwraca inne topiki, niż poprzedni zestaw parametrów.

In [7]:
lda_passes = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=10, update_every=0, passes=50)
lda_passes.print_topics(10)

[(0,
  '0.022*"may" + 0.020*"game" + 0.019*"us" + 0.012*"kurt" + 0.011*"cobain" + 0.009*"5th" + 0.009*"thrones" + 0.008*"know" + 0.007*"win" + 0.007*"come"'),
 (1,
  '0.023*"apple" + 0.019*"will" + 0.018*"may" + 0.015*"watch" + 0.010*"like" + 0.010*"juventus" + 0.010*"&amp;" + 0.008*"new" + 0.008*"barca" + 0.008*"3rd"'),
 (2,
  '0.015*"going" + 0.014*"may" + 0.014*"bobby" + 0.013*"?" + 0.011*"conor" + 0.011*"just" + 0.011*"tomorrow" + 0.010*"mcgregor" + 0.010*"jindal" + 0.009*"2nd"'),
 (3,
  '0.019*"tomorrow" + 0.017*"angela" + 0.014*"merkel" + 0.012*"see" + 0.012*"-" + 0.009*"ihop" + 0.009*"just" + 0.009*"still" + 0.008*"1st" + 0.008*"like"'),
 (4,
  '0.019*"may" + 0.017*"mike" + 0.016*"magic" + 0.016*"new" + 0.014*"david" + 0.012*"beckham" + 0.012*"just" + 0.012*"xxl" + 0.010*"harry" + 0.009*"tomorrow"'),
 (5,
  '0.016*"eric" + 0.015*"chris" + 0.015*"grateful" + 0.014*"church" + 0.013*"dead" + 0.013*"brown" + 0.012*"going" + 0.012*"will" + 0.009*"may" + 0.009*"day"'),
 (6,
  '0.035*"

#### Zwiększenie `num_topics` do 15
Zwiększenie `num_topics` zmusiło algorytm do znalezienia większej liczby topików. Dlatego wynik zawiera więcej elementów. Widać też, że nowe topicki nie pokrywają się ze starymy, więc nie znalazł po prostu 5 nowych, tylko 15 nowych od zera.

In [8]:
lda_topics = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=15, update_every=0, passes=20)
lda_topics.print_topics(15)

[(0,
  '0.021*"mike" + 0.021*"magic" + 0.015*"fleetwood" + 0.015*"xxl" + 0.015*"may" + 0.013*"&amp;" + 0.012*"mac" + 0.010*"1st" + 0.009*"just" + 0.009*"-"'),
 (1,
  '0.022*"may" + 0.015*"angela" + 0.014*"merkel" + 0.010*"will" + 0.008*"new" + 0.007*"birthday" + 0.007*"happy" + 0.007*"2nd" + 0.007*"chelsea" + 0.006*"&amp;"'),
 (2,
  '0.012*"ipad" + 0.010*"1st" + 0.010*"may" + 0.009*"still" + 0.008*"will" + 0.008*"need" + 0.008*"apple" + 0.007*"it." + 0.007*"get" + 0.006*"just"'),
 (3,
  '0.021*"may" + 0.016*"tomorrow" + 0.012*"-" + 0.011*"friday" + 0.010*"apple" + 0.010*"grateful" + 0.009*"day" + 0.009*"dead" + 0.009*"will" + 0.008*"eric"'),
 (4,
  '0.025*"harry" + 0.022*"potter" + 0.018*"chris" + 0.017*"tomorrow" + 0.017*"brown" + 0.014*"going" + 0.011*"may" + 0.010*"disneyland" + 0.010*"get" + 0.008*"see"'),
 (5,
  '0.039*"may" + 0.021*"trump" + 0.019*"donald" + 0.014*"jeb" + 0.013*"will" + 0.012*"bush" + 0.010*"google" + 0.010*"bernie" + 0.009*"like" + 0.009*"sanders"'),
 (6,
  '0.0

**Zad. 6*: Na podstawie zbudowanego modelu określ proporcje tematów w następującym tweecie:
`Zlatan is looking mighty attractive at the moment,if LVG doesn't get a striker by Tuesday, I really don't fancy us scoring goals this season`. Jeśli zostało niewiele czasu, przejdź od razu do wizualizacji.**

In [9]:
new_tweet = "Zlatan is looking mighty attractive at the moment,if LVG doesn't get a striker by Tuesday, I really don't fancy us scoring goals this season"
new_tweet_text = [word for word in new_tweet.lower().split() if word not in stopwords and counter[word] > 1]
new_tweet_mm = id2word.doc2bow(new_tweet_text)
new_tweet_lda = lda[new_tweet_mm]
print(new_tweet_lda)

[(0, 0.20808348), (1, 0.010004786), (2, 0.7118807), (3, 0.010005019), (4, 0.010005161), (5, 0.010003465), (6, 0.010004338), (7, 0.010005064), (8, 0.010003979), (9, 0.010003976)]


## Wizualizacja

Spróbujemy teraz zwizualizować uzyskane tematy. Podążaj za komentarzami a powinno się udać.

P.S. Możesz uruchomić tę wizualizację również dla tematów odkrytych z Wikipedii...

**Zad. 7: Zwizualizuj uzyskane tematy.**

In [10]:
# # Poniższy kod korzysta modułu pyLDAvis z githuba:
# # https://github.com/bmabey/pyLDAvis

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, mm, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.024845  0.024018       1        1  11.554677
6      0.115315 -0.059091       2        1  10.950795
3      0.081421  0.009954       3        1  10.554397
7      0.050673  0.045866       4        1  10.473066
2      0.085559  0.097669       5        1  10.450215
5     -0.050484  0.000439       6        1  10.396871
1     -0.181392  0.040155       7        1  10.296512
4     -0.084943 -0.012203       8        1   9.112871
9     -0.028621  0.050509       9        1   8.672367
8     -0.012372 -0.197317      10        1   7.538228, topic_info=           Term        Freq       Total Category  logprob  loglift
14    available  478.000000  478.000000  Default  30.0000  30.0000
5501   jurassic  170.000000  170.000000  Default  29.0000  29.0000
1007      apple  235.000000  235.000000  Default  28.0000  28.0000
917      amazon  242.000000  242.000000  Default  27.0000  27.0000
243       prime  167.000000  167.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
23         will   17.080968  406.482594  Topic10  -5.4634  -0.5844
69          get   15.157160  247.767296  Topic10  -5.5828  -0.2088
184         one   13.520895  174.117316  Topic10  -5.6971   0.0297
95            -   14.099314  349.129490  Topic10  -5.6552  -0.6241
3960      david   12.966108  165.955429  Topic10  -5.7390   0.0358

[751 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3240      1  0.737554         "the
3240      8  0.122926         "the
3130      8  0.826884            #
1966      2  0.935895       #apple
1907      2  0.950632  #appleevent
...     ...       ...          ...
966       4  0.049047            |
966       6  0.049047            |
966       7  0.343327            |
966       9  0.098093            |
966      10  0.392374            |

[1977 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 4, 8, 3, 6, 2, 5, 10, 9])